In [1]:
from classes import Trip
from flights import Flights
from amadeus import Client, ResponseError
from datetime import timedelta, date
import datetime
from hotels1 import hotelFunctions
from transport import transportFunctions
from transport import getNZfromCity
from amadeus import Location
import random
from classes import Day

def cleanUsedAttractions(attractions, usedAttractions):
    newAttractionList = []
    for i in attractions:
        if (i.title.lower() in usedAttractions):
            continue
        newAttractionList.append(i)
    return newAttractionList

def getLandingDate(initFlight):
    relevantFlight = initFlight[-1]
    return relevantFlight.timeEnd
def getDepartureDate(finFlight):
    relevantFlight = finFlight[0]
    return relevantFlight.timeStart

def chooseCheapestHotel(hotels):
    lowest = 1000000
    index = 0
    for i in range(0, len(hotels)):
        if (hotels[i].cost < lowest):
            index = i
            lowest = hotels[i].cost
    return hotels[index]
def chooseMostLuxuriousHotel(hotels):
    highest = 0
    index = 0
    for i in range(0, len(hotels)):
        if (hotels[i].cost > highest):
            index = i
            highest = hotels[i].cost
    return hotels[index]
def chooseHotel(lowCost, luxury, hotelObject, city, passengerCount):
    print(getNZfromCity(city)[0], getNZfromCity(city)[1])
    xCoord = getNZfromCity(city)[0]
    yCoord = getNZfromCity(city)[1]
    hotels = []
    while (hotels == []):
        hotels = hotelObject.getHotelsByGeocode(xCoord, yCoord)
        print("Hotel results are", hotels, xCoord, yCoord)
        xCoord += (random.random() - 0.5) * 0.4
        yCoord += (random.random() - 0.5) * 0.4
    print(hotels)
    for i in hotels:
        print(i)
    choosenHotel = None
    if (lowCost):
        choosenHotel = chooseCheapestHotel(hotels)
    elif (luxury):
        choosenHotel = chooseMostLuxuriousHotel(hotels)
    else:
        choosenHotel = random.choice(hotels)
    choosenHotel.cost = choosenHotel.cost * passengerCount
    return choosenHotel

def planDay(fastPaced, attractionOptions, transportObject, xCoord, yCoord, date, usedAttractions, passengerCount, hotel, currDate = [], attractionCount = 2):
    if (currDate == []):
        currDate = date.replace(minute = 0, hour = 9, second=0)
    startingDate = currDate
    print("THE DATE IS:", currDate)
    attractionArr = []
    transportArr = []
    currX = xCoord
    currY = yCoord
    if (fastPaced and attractionCount == 2):
        attractionCount = 3
    for i in range(0, min((attractionCount), len(attractionOptions))):
        attractionLocation = attractionOptions[i].destination.split(',')
        destX = float(attractionLocation[0])
        destY = float(attractionLocation[1])
        currTransport = transportObject.getTransportByTime(currX, currY, destX, destY, str(currDate.isoformat()))
        currX = destX
        currY = destY
        print(currTransport)
        currDate = currTransport[-1].timeEnd
        currDate = currDate + datetime.timedelta(minutes=5)
        transportArr.append(currTransport)
        attractionOptions[i].timeStart = currDate
        currDate = currDate + datetime.timedelta(hours=3)
        attractionOptions[i].timeEnd = currDate
        currDate = currDate + datetime.timedelta(minutes=60)
        attractionOptions[i].cost = attractionOptions[i].cost * passengerCount
        attractionArr.append(attractionOptions[i])
        usedAttractions.append(attractionOptions[i].title.lower())
    print("The date at the end is", currDate)
    currTransport = transportObject.getTransportByTime(currX, currY, xCoord, yCoord, str(currDate.isoformat()))
    for trans in currTransport:
        if (trans.cost != None):
            trans.cost = trans.cost * passengerCount
    transportArr.append(currTransport)
    print("Found Attractions:\n\n\n")
    for i in attractionArr:
        print(i)
    print("Found Trip\n\n\n")
    for i in transportArr:
        for j in i:
            print(j)
    return Day(attractionArr, transportArr, 0, startingDate, currTransport[-1].timeEnd, 0, hotel), usedAttractions

def planFullDay(tripPeople, luxury, lowCost, museumOriented, 
                fastPaced, priceCapLeft, attractionObject, transportObject, hotel, usedAttractions, date):
    xCoord = hotel.destination.split(',')
    originalYCoord = float(xCoord[1])
    originalXCoord = float(xCoord[0])
    xCoord = originalXCoord
    yCoord = originalYCoord
    print(xCoord, yCoord)
    attractions = [] # maybe add most_central_location implementation later
    while (attractions == [] or len(attractions) < 3):
        attractions = attractionObject.getActivities(xCoord, yCoord, 10)
        attractions = cleanUsedAttractions(attractions, usedAttractions)
        noise = (random.random() - 0.5) * 0.4
        xCoord += noise
        yCoord += noise
    attractionOptions = []
    for i in attractions:
        print(i)
    if (museumOriented):
        for i in attractions:
            if ('museum' in i.title.lower()):
                attractionOptions.append(i)
        if (attractionOptions == []):
            attractionOptions = attractions
            noise = (random.random() - 0.5) * 0.2
            xCoord += noise
            yCoord += noise
    else:
        attractionOptions = attractions
    if (lowCost):
        attractionOptions.sort(key=lambda x: x.cost)
    elif (luxury):
        attractionOptions.sort(key=lambda x: x.cost, reverse=True)
    day, usedAttractions = planDay(fastPaced, attractionOptions, transportObject, originalXCoord, originalYCoord, date, usedAttractions, tripPeople, hotel)
    return day, usedAttractions

def createArrivalDay(airportCoords, hotel, amadeusObject, arrivalTime, usedAttractions, hotelObject, luxury, lowCost, museumOriented, fastPace, passengerCount):
    coords = hotel.destination.split(',')
    hotelXCoord = coords[0]
    hotelYCoord = coords[1]
    arrivalTime = arrivalTime + datetime.timedelta(hours=2) 
    exitTime = arrivalTime.isoformat()
    transportObject = transportFunctions()
    airportToHotel = transportObject.getTransportByTime(airportCoords[0], airportCoords[1], hotelXCoord, hotelYCoord, exitTime)
    for trans in airportToHotel:
        print(trans)
        if (trans.cost != None):
            trans.cost = trans.cost * passengerCount
    hotelArrivalTime = airportToHotel[-1].timeEnd
    print(hotelArrivalTime)
    
    hotelArrivalTime += datetime.timedelta(hours=2)
    if (hotelArrivalTime.time().hour < 18):
        hotelArrivalTime = hotelArrivalTime.replace(hour = max(hotelArrivalTime.time().hour, 9))
        attractions = hotelObject.getActivities(hotel.destination.split(',')[0], hotel.destination.split(',')[1], 2)
        plannedDay, usedAttractions = planDay(False, attractions, transportObject, hotelXCoord, hotelYCoord, hotelArrivalTime, [], passengerCount, hotel, currDate=hotelArrivalTime, attractionCount = 1)
        plannedDay.transportation.append(airportToHotel)
    else:
        usedAttractions = []
        plannedDay = Day([], [airportToHotel], 0, arrivalTime, airportToHotel[-1].timeEnd, 0, hotel)
    print("In create arrival day:")
    print(airportToHotel)
    for i in airportToHotel:
        print(i)
    return plannedDay, usedAttractions
def createDepartureDay(airportCoords, hotel, amadeusObject, departureTime, usedAttractions, hotelObject, luxury, lowCost, museumOriented, fastPace, passengerCount):
    coords = hotel.destination.split(',')
    hotelXCoord = coords[0]
    hotelYCoord = coords[1]
    departureTime = departureTime + datetime.timedelta(hours=6) 
    exitTime = departureTime.isoformat()
    transportObject = transportFunctions()
    hotelToAirport = transportObject.getTransportByTime(hotelXCoord, hotelYCoord, airportCoords[0], airportCoords[1], exitTime)
    for trans in hotelToAirport:
        if (trans.cost != None):
            trans.cost = trans.cost * passengerCount
    airportArrivalTime = hotelToAirport[-1].timeEnd
    plannedDay = Day([], [hotelToAirport], 0, exitTime, departureTime, 0, hotel)

In [2]:
def getTrip(srcAirport, date, duration, passengerCount, isFastPaced, isMuseumOriented, isLuxury, isLowCost, city):
    amadeusBaseObject = Client(
    client_id='wv6i3ROKO0i2d0sitNtjN4pTKTynYZZa',
    client_secret='ZzSCP9nj8AA35LqV'
    )
    duration -= 1
    flightObject = Flights(amadeusBaseObject)
    nearestAirport = amadeusBaseObject.reference_data.locations.get(
    keyword=city,
    subType=Location.ANY
    )
    nearestAirport = nearestAirport.data
    destinationAirport = nearestAirport[0]['iataCode']
    coordsArr = [nearestAirport[0]['geoCode']['latitude'], nearestAirport[0]['geoCode']['longitude']]
    initFlight = flightObject.getFlightObjects(srcAirport, destinationAirport, date, passengerCount)
    landingDate = str(getLandingDate(initFlight)).split(' ')[0]
    dateFormat = '%Y-%m-%d' 
    dateObject = datetime.datetime.strptime(landingDate, dateFormat)
    finFlight = flightObject.getFlightObjects(destinationAirport, srcAirport, (dateObject + timedelta(days=duration)).strftime("%Y-%m-%d"), passengerCount)
    tripStartTime = getLandingDate(initFlight)
    tripEndTime = getDepartureDate(finFlight)
    hotelObject = hotelFunctions(amadeusBaseObject)
    hotel = chooseHotel(isLowCost, isLuxury, hotelObject, city, passengerCount)
    print(hotel)
    attractions = hotelObject.getActivities(hotel.destination.split(',')[0], hotel.destination.split(',')[1], 2)
    currDate = tripStartTime
    usedAttractions = []
    dayArr = []
    while (currDate.date() != tripEndTime.date()):
        print("THIS IS THE DATES", currDate.date(), tripEndTime.date())
        if (currDate == tripStartTime):
            results = createArrivalDay(coordsArr, hotel, amadeusBaseObject, tripStartTime, usedAttractions, hotelObject, isLuxury, isLowCost, isMuseumOriented, isFastPaced, passengerCount)
            print("Creating arrival day")
            dayArr.append(results[0])
            usedAttractions = results[1]
        else:
            transport = transportFunctions()
            results = planFullDay(passengerCount, isLuxury, isLowCost, isMuseumOriented, isFastPaced, 10000, hotelObject, transport, hotel, usedAttractions, currDate)
            print("Creating regular day")
            dayArr.append(results[0])
            usedAttractions = results[1]
        currDate = currDate + timedelta(days=1)
    print(attractions)
    lastDay = createDepartureDay(coordsArr, hotel, amadeusBaseObject, tripEndTime, usedAttractions, hotelObject, isLuxury, isLowCost, isMuseumOriented, isFastPaced, passengerCount)
    dayArr.append(lastDay)
    tripObject = Trip("Tripping like a trip", city, duration, initFlight[0].timeStart, finFlight[-1].timeEnd, dayArr, 0, '5', initFlight, finFlight)
    print(tripObject)
    return tripObject

In [3]:
getTrip('MAD', '2022-07-07', 5, 4, True, False, True, False, 'New York')


{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2022-07-07', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT29H55M', 'segments': [{'departure': {'iataCode': 'MAD', 'terminal': '4S', 'at': '2022-07-07T19:45:00'}, 'arrival': {'iataCode': 'CMN', 'terminal': '2', 'at': '2022-07-07T20:35:00'}, 'carrierCode': 'AT', 'number': '971', 'aircraft': {'code': '73H'}, 'operating': {'carrierCode': 'AT'}, 'duration': 'PT2H50M', 'id': '262', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'CMN', 'terminal': '1', 'at': '2022-07-08T15:50:00'}, 'arrival': {'iataCode': 'JFK', 'at': '2022-07-08T18:40:00'}, 'carrierCode': 'AT', 'number': '200', 'aircraft': {'code': '789'}, 'operating': {'carrierCode': 'AT'}, 'duration': 'PT7H50M', 'id': '263', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '1503.12', 'base': '484.00', 'fees': [

{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2022-07-11', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT9H10M', 'segments': [{'departure': {'iataCode': 'EWR', 'at': '2022-07-11T21:50:00'}, 'arrival': {'iataCode': 'LHR', 'at': '2022-07-12T08:45:00'}, 'carrierCode': '6X', 'number': '188', 'aircraft': {'code': '777'}, 'operating': {'carrierCode': '6X'}, 'duration': 'PT5H55M', 'id': '27', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'LHR', 'at': '2022-07-12T10:30:00'}, 'arrival': {'iataCode': 'MAD', 'at': '2022-07-12T13:00:00'}, 'carrierCode': '6X', 'number': '9931', 'aircraft': {'code': '320'}, 'operating': {'carrierCode': '6X'}, 'duration': 'PT1H30M', 'id': '28', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '1303.36', 'base': '1128.00', 'fees': [{'amount': '0.00', 'type': 'SUPPLIER'}, {'amount': '0

40.68295 -73.9708
Hotel results are [<classes.PlaceOfStay object at 0x000002AB084122B0>, <classes.PlaceOfStay object at 0x000002AB08349E50>, <classes.PlaceOfStay object at 0x000002AB08412340>, <classes.PlaceOfStay object at 0x000002AB083699D0>, <classes.PlaceOfStay object at 0x000002AB08369670>, <classes.PlaceOfStay object at 0x000002AB083C92B0>, <classes.PlaceOfStay object at 0x000002AB083C9670>, <classes.PlaceOfStay object at 0x000002AB08414790>, <classes.PlaceOfStay object at 0x000002AB08414190>] 40.68295 -73.9708
[<classes.PlaceOfStay object at 0x000002AB084122B0>, <classes.PlaceOfStay object at 0x000002AB08349E50>, <classes.PlaceOfStay object at 0x000002AB08412340>, <classes.PlaceOfStay object at 0x000002AB083699D0>, <classes.PlaceOfStay object at 0x000002AB08369670>, <classes.PlaceOfStay object at 0x000002AB083C92B0>, <classes.PlaceOfStay object at 0x000002AB083C9670>, <classes.PlaceOfStay object at 0x000002AB08414790>, <classes.PlaceOfStay object at 0x000002AB08414190>]
{'Durati

{'routes': [{'id': 'C0', 'sections': [{'id': 'C0-S0', 'type': 'pedestrian', 'departure': {'time': '2022-07-08T09:04:00-04:00', 'place': {'type': 'place', 'location': {'lat': 40.70541, 'lng': -74.00783}}}, 'arrival': {'time': '2022-07-08T09:05:00-04:00', 'place': {'name': 'Water St/Wall St', 'type': 'station', 'location': {'lat': 40.705045, 'lng': -74.007758}, 'id': '717061284'}}, 'transport': {'mode': 'pedestrian'}}, {'id': 'C0-S1', 'type': 'transit', 'departure': {'time': '2022-07-08T09:05:00-04:00', 'place': {'name': 'Water St/Wall St', 'type': 'station', 'location': {'lat': 40.705045, 'lng': -74.007758}, 'id': '717061284'}}, 'arrival': {'time': '2022-07-08T09:08:00-04:00', 'place': {'name': 'South Ferry/Terminal', 'type': 'station', 'location': {'lat': 40.70191, 'lng': -74.013846}, 'id': '717061712'}}, 'transport': {'mode': 'bus', 'name': 'M15-SBS', 'category': 'Bus', 'color': '#006CB7', 'textColor': '#FFFFFF', 'headsign': 'SELECT BUS SERVICE SOUTH FERRY via 2 AV', 'shortName': 'M15

{'routes': [{'id': 'C0', 'sections': [{'id': 'C0-S0', 'type': 'pedestrian', 'departure': {'time': '2022-07-09T09:11:00-04:00', 'place': {'type': 'place', 'location': {'lat': 40.70541, 'lng': -74.00783}}}, 'arrival': {'time': '2022-07-09T09:13:00-04:00', 'place': {'name': 'Water St/Gouverneur LA', 'type': 'station', 'location': {'lat': 40.704661, 'lng': -74.008294}, 'id': '717061643'}}, 'transport': {'mode': 'pedestrian'}}, {'id': 'C0-S1', 'type': 'transit', 'departure': {'time': '2022-07-09T09:13:00-04:00', 'place': {'name': 'Water St/Gouverneur LA', 'type': 'station', 'location': {'lat': 40.704661, 'lng': -74.008294}, 'id': '717061643'}}, 'arrival': {'time': '2022-07-09T09:16:00-04:00', 'place': {'name': 'South St/Whitehall St', 'type': 'station', 'location': {'lat': 40.701609, 'lng': -74.012226}, 'id': '717061698'}}, 'transport': {'mode': 'bus', 'name': 'M15', 'category': 'Bus', 'color': '#006CB7', 'textColor': '#FFFFFF', 'headsign': 'SOUTH FERRY via 2 AV', 'shortName': 'M15', 'longN

{'Duration': None, 'Cost': 26.0, 'Times:': [None, None], 'Title': '9-11 Memorial and Museum tickets', 'Dest': '40.711548,-74.013292'}
{'Duration': None, 'Cost': 36.0, 'Times:': [None, None], 'Title': 'NYC One World Observatory Skip-the-Line Ticket', 'Dest': '40.711544,-74.013487'}
{'Duration': None, 'Cost': 39.0, 'Times:': [None, None], 'Title': 'Empire State Building Tickets - Observatory and Optional Skip the Line Tickets', 'Dest': '40.712784,-74.005941'}
{'Duration': None, 'Cost': 42.0, 'Times:': [None, None], 'Title': 'Circle Line: Complete Manhattan Island Cruise', 'Dest': '40.712784,-74.005941'}
{'Duration': None, 'Cost': 39.0, 'Times:': [None, None], 'Title': 'Top of the Rock Observation Deck, New York', 'Dest': '40.712784,-74.005941'}
{'Duration': None, 'Cost': 27.0, 'Times:': [None, None], 'Title': '9/11 Memorial Museum Admission', 'Dest': '40.711626,-74.010714'}
{'Duration': None, 'Cost': 33.0, 'Times:': [None, None], 'Title': '9/11 Memorial and Ground Zero Tour with Optional

{'routes': [{'id': 'C0', 'sections': [{'id': 'C0-S0', 'type': 'pedestrian', 'departure': {'time': '2022-07-12T00:10:00-04:00', 'place': {'type': 'place', 'location': {'lat': 40.70541, 'lng': -74.00783}}}, 'arrival': {'time': '2022-07-12T00:14:00-04:00', 'place': {'name': 'Wall St', 'type': 'station', 'location': {'lat': 40.706821, 'lng': -74.0091}, 'id': '717080193'}}, 'transport': {'mode': 'pedestrian'}}, {'id': 'C0-S1', 'type': 'transit', 'departure': {'time': '2022-07-12T00:14:00-04:00', 'place': {'name': 'Wall St', 'type': 'station', 'location': {'lat': 40.706821, 'lng': -74.0091}, 'id': '717080193'}}, 'arrival': {'time': '2022-07-12T00:18:00-04:00', 'place': {'name': 'Chambers St', 'type': 'station', 'location': {'lat': 40.715478, 'lng': -74.009266}, 'id': '717080103'}}, 'transport': {'mode': 'subway', 'name': '2', 'category': 'Subway, Metro', 'color': '#EE352E', 'headsign': 'Wakefield-241 St', 'shortName': '2', 'longName': '7 Avenue Express'}, 'agency': {'id': '2GzMTA00', 'name':